In [ ]:
%pylab inline
from pyspark.sql.types import *
from datetime import datetime
from pyspark.sql import Row
from time import time

# Introduction

Spark will usually split your data into a set of partitions automatically but there are cases where you want to do this manually to improve performance. You can us the repartition() method to define the number of partitions in your RDD. In this tutorial we explore the number of partitions on performance.

We are going to be using the Reddit comments dataset for this tutorial. More information on this dataset can be found [here](https://sites.google.com/a/insightdatascience.com/spark-lab/s3-data/reddit-comments).

# Load in the data

In [ ]:
fields = [StructField("archived", BooleanType(), True),
        StructField("author", StringType(), True),
        StructField("author_flair_css_class", StringType(), True),
        StructField("body", StringType(), True),
        StructField("controversiality", LongType(), True),
        StructField("created_utc", StringType(), True),
        StructField("day", LongType(), True),
        StructField("distinguished", StringType(), True),
        StructField("downs", LongType(), True),
        StructField("edited", StringType(), True),
        StructField("gilded", LongType(), True),
        StructField("id", StringType(), True),
        StructField("link_id", StringType(), True),
        StructField("month", LongType(), True),
        StructField("name", StringType(), True),
        StructField("parent_id", StringType(), True),
        StructField("retrieved_on", LongType(), True),
        StructField("score", LongType(), True),
        StructField("score_hidden", BooleanType(), True),
        StructField("subreddit", StringType(), True),
        StructField("subreddit_id", StringType(), True),
        StructField("ups", LongType(), True),
        StructField("year", LongType(), True)]
#rawDF = sqlContext.read.json("s3a://reddit-comments/2008", StructType(fields))
rawDF = sqlContext.read.parquet("s3a://reddit-comments-parquet/year=2008")
rawDF.persist(StorageLevel.DISK_ONLY)
rawDF.count()

# Parallelism - example 1

To analyze the performance of a simple Spark job with different numbers of partitions, let's split our data into varying numbers of partitions. We'll persist the partitions to save recomputing down the road.

In [ ]:
# Setting up DataFrames to have various number of partitions ranging from 2 to 32
# All will be persisted on disk and the same job will be run on each one to show 
# performance benefits of different number of partitions
repart_1000_df = rawDF.repartition(1000).persist(StorageLevel.DISK_ONLY)
repart_2000_df = rawDF.repartition(2000).persist(StorageLevel.DISK_ONLY)
repart_4000_df = rawDF.repartition(4000).persist(StorageLevel.DISK_ONLY)
repart_8000_df = rawDF.repartition(8000).persist(StorageLevel.DISK_ONLY)
repart_16000_df = rawDF.repartition(16000).persist(StorageLevel.DISK_ONLY)


Now let's perform a simple job (count the number of elements in each dataframe) and compare how long the operation takes for various number of partitions.

In [ ]:
# Run through each DataFrame and count the number of elements in each. This will 
# trigger the DataFrames to persist into Memory and Disk
df_arr = [(rawDF, rawDF.rdd.getNumPartitions()),
          (repart_1000_df, repart_1000_df.rdd.getNumPartitions()),
          (repart_2000_df, repart_2000_df.rdd.getNumPartitions()),
          (repart_4000_df, repart_4000_df.rdd.getNumPartitions()),
          (repart_8000_df, repart_8000_df.rdd.getNumPartitions()),
          (repart_16000_df, repart_16000_df.rdd.getNumPartitions())]
for df in df_arr:
    start_time = time()
    df[0].count()
    end_time = time()
    print "{} partitions took {} seconds to repartition and count".format(df[1], end_time - start_time) 

### What does the runtime indicate about the scheduling and distribution of tasks to worker nodes?

### Look at 4040 to see how the count action varied in time with increased number of partitions

# Parallelism - example 2

In [ ]:
# Function sorts an array of UTC time and calculates the delta time in days between each consecutive UTC time
# Returns a Row object containing the subreddit, the median time delta representing the median time in days 
# between comments for any subreddit, and the total number of comments in each subreddit
def calc_median(row):
    from heapq import heappop
    from numpy import median
    
    subreddit = row[0]
    val_arr = row[1]
    num_comments = len(val_arr)
    
    dt = []

    if len(val_arr) > 1:
        prev_val = heappop(val_arr)
        while len(val_arr) > 0:
            curr_val = heappop(val_arr)
            dt.append(curr_val - prev_val)
            prev_val = curr_val
        return Row(subreddit=subreddit, median_time_days=float(median(dt))/60.0/60.0/24.0, num_comments=num_comments)
    else:
        return Row(subreddit=subreddit, median_time_days=0.0, num_comments=num_comments)
        
    

### The following code is very inefficient. Can you figure out why (before attempting to run it)?

In [ ]:
# Loop through each DataFrame and run the same job to calculate the median time between comments for each
# subreddit
def combiner(value):
    return value

def merger(x, value):
    from heapq import heappush

    heappush(x, value[0])

    return x

def merge_combiner(x, y):
    from heapq import heappush, heappop

    while len(y) > 0:
        heappush(x, heappop(y))

    return x

results = []
for df in df_arr:
    start_time = time()
    curr_df = df[0]
    num_partitions = df[1]

    subreddit_comment_times = curr_df.rdd.map(lambda r: (r.subreddit, [int(r.created_utc)]))
    median_time_between_posts_df = subreddit_comment_times.combineByKey(combiner, merger, merge_combiner)\
                                                          .map(calc_median)\
                                                          .toDF()

    total_cnt = median_time_between_posts_df.count()
    end_time = time()
    print "{} partitions took {} seconds to process, {} final records".format(df[1], end_time - start_time, total_cnt) 

In [ ]:
# A scatter plot where each element is a subreddit. We can see that subreddits with more than 40 comments tend to
# also have more frequent comment activity throughout the year
median_time_between_posts_pd = median_time_between_posts_df.toPandas()
median_time_between_posts_pd.plot(kind='scatter', x='num_comments', y='median_time_days', xlim=(-5, 100), ylim=(-5, 100))

# Next Steps

## Task 1: What is the optimal number of partitions for calculating the author who has written the longest comment?